In [5]:
import pandas as pd
import json
import re
import os
from pathlib import Path

satellite_path = Path(os.getcwd()).parent.parent / "data" / "excel" / "спутники.xlsx"

satellite_names = [
    "ASTER",
    "LANDSAT-89 OLITIRS",
    "Landsat-7 ETM+",
    "Landsat-45 TM",
    "Landsat-123 RBV",
    "Sentinel-2A MSI",
    "EMIT",
    "EnMAP",
    "EO-1 Hyperion",
]

def parse_satellite_bands_table(
    data_path: Path,
    sheet_list: list[str]
) -> dict:
    satellite_data = pd.ExcelFile(satellite_path)
    satellite_bands = {}
    
    for sat in sheet_list:
        data = {}
        try:
            df = pd.read_excel(satellite_path ,sheet_name = sat)
            for a, d in zip(df['Алиас'],df['Диапазон, нм']):
                if type(a) != str:
                    continue
                bounds = [
                    round(float(d.split("-")[0]), 2),
                    round(float(d.split("-")[1]), 2)
                ]
                data[a] = bounds
        except:
            print(f"Не удалось спарсить лист: {sat}")
        satellite_bands[sat] = data
    return satellite_bands


In [23]:
df = pd.ExcelFile(satellite_path)
df.sheet_names

['Обзор экспорта',
 'ASTER',
 'LANDSAT-89 OLITIRS',
 'Landsat-7 ETM+',
 'Landsat-45 TM',
 'Landsat-12345 MSS',
 'Landsat-123 RBV',
 'Sentinel-2A MSI',
 'Sentinel-2B MSI',
 'Sentinel-2C MSI',
 'EMIT',
 'EnMAP',
 'EO-1 Hyperion',
 'EO-1 ALI',
 'EO-1 LAC',
 'Prisma',
 'Канопус-В ПСС',
 'Канопус-В №1 и Канопус-В-ИК МСС',
 'Канопус-В 3456 МСС',
 'Канопус-В-ИК МСУ-ИК-СРМ',
 'Ресурс-П СППИ &quot;Сангур-1У&q',
 'Ресурс-П КШМСА ШМСА-ВР',
 'Ресурс-П КШМСА ШМСА-СР',
 'Ресурс-П ГСА']

# **Скачивание статей**

In [18]:
index_table_path = Path(os.getcwd()).parent.parent / "data" / "excel" / "FilteredFormulas.xlsx"
df = pd.read_excel(index_table_path)


In [9]:
ALIASES = {
    "BLUE": (450, 520),
    "B":    (450, 520),
    "GREEN": (520, 600),
    "G":     (520, 600),
    "RED": (630, 690),
    "R":   (630, 690),
    "NIR": (760, 1400),
    "MIR": (1500,1750),
    "SWIR": (1400, 3000),
    "SWIR1": (1550, 1750),
    "SWIR2": (2080, 2350),
    "MIR": (3000, 5000),
    "rededge": (700, 720)
}


def alias_to_range(name: str):
    """Return alias numeric center value or (lo,hi)."""
    if name not in ALIASES:
        return None
    lo, hi = ALIASES[name]

    return (lo + hi) / 2   # можно вернуть диапазон полностью, если хочешь


def preprocess_aliases(formula: str) -> str:
    """Replace spectral alias names (RED, NIR, etc.) with numeric values."""
    for name in ALIASES:
        pattern = r"\b" + name + r"\b"
        value = alias_to_range(name)
        formula = re.sub(pattern, f"[{ALIASES[name][0]}:{ALIASES[name][1]}]", formula)
    return formula

In [10]:
preprocess_aliases("((2*(NIR**2 - RED**2) + 1.5*NIR + 0.5*RED )/(NIR+RED+0.5))*(1-0.25*((2*(NIR**2-RED**2) + 1.5*NIR+0.5*RED)/(NIR+RED+0.5))) - (660nm - 0.125)/(1 - 660nm)")

'((2*([760:1400]**2 - [630:690]**2) + 1.5*[760:1400] + 0.5*[630:690] )/([760:1400]+[630:690]+0.5))*(1-0.25*((2*([760:1400]**2-[630:690]**2) + 1.5*[760:1400]+0.5*[630:690])/([760:1400]+[630:690]+0.5))) - (660nm - 0.125)/(1 - 660nm)'

In [12]:
import re
import sympy as sp
from sympy import sympify, simplify

def wavelength_to_band(wl_min, wl_max, bands):
    for bname, (bmin, bmax) in bands.items():
        # intersection check
        if wl_min >= bmin and wl_max <= bmax:
            return bname
    return "nan"


def convert_formula(formula, sat="landsat8"):
    f = formula

    # --- Replace ranges [700:710] ---
    def repl_range(m):
        lo = int(m.group(1))
        hi = int(m.group(2))
        b = wavelength_to_band(lo, hi, sat)
        return b

    f = re.sub(r"\[(\d+):(\d+)\]", repl_range, f)

    # --- Replace single [700] ---
    def repl_single(m):
        wl = int(m.group(1))
        b = wavelength_to_band(wl, wl, sat)
        return b

    f = re.sub(r"\[(\d+)\]", repl_single, f)

    # --- Replace "700nm" ---
    def repl_nm(m):
        wl = int(m.group(1))
        b = wavelength_to_band(wl, wl, sat)
        return b

    f = re.sub(r"(\d+)nm", repl_nm, f)

    # If variables like RED, BLUE appear — leave as is.

    # --- Simplify with sympy ---
    try:
        expr = sympify(f)
        simplified = simplify(expr)
        
        # Проверяем, является ли результат константой
        if simplified.is_constant():
            return "constant"
        elif 'zoo' in str(simplified):
            return "nan"
        else:
            f = str(simplified)
    except Exception:
        pass

    return f



In [15]:


convert_formula('((2*([760:1400]**2 - [630:690]**2) + 1.5*[760:1400] + 0.5*[630:690] )/([760:1400]+[630:690]+0.5))*(1-0.25*((2*([760:1400]**2-[630:690]**2) + 1.5*[760:1400]+0.5*[630:690])/([760:1400]+[630:690]+0.5))) - (660nm - 0.125)/(1 - 660nm)', )

AttributeError: 'str' object has no attribute 'items'

In [118]:
for sat in satellite_names:
    df[sat] = df['formula'].apply(lambda x: convert_formula(x, satellite_bands[sat]))



In [119]:
df.to_excel("index_table_v2.xlsx", index=False)

In [113]:
convert_formula("700nm + 40*(670nm/2 - 700nm + 780nm/2)/(-700nm + 740nm)", satellite_bands['Landsat-7 ETM+'])

'SR_B8 + zoo*(SR_B3 + SR_B4 - 2*SR_B8)'

In [109]:
satellite_bands['Landsat-7 ETM+']

{'SR_B1': [450.0, 520.0],
 'SR_B2': [520.0, 600.0],
 'SR_B3': [630.0, 690.0],
 'SR_B4': [770.0, 900.0],
 'SR_B5': [1550.0, 1750.0],
 'ST_B6': [10400.0, 12500.0],
 'SR_B7': [2090.0, 2350.0],
 'SR_B8': [520.0, 900.0]}

In [ ]:
def generate_excel_formulas(
    index_table_path: str,
    satellite_path: str,
    output_path: str
):
    
    